<b>Version 1.0 LAC做NER 計算Group NER 及 PER 的詞頻<b>

In [2]:
import re
import requests
import math
from lxml import html
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 匯入資料

In [ ]:
def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
#         print(idx, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
        except Exception as e:
            print(e)
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text, getLink

def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [ ]:
## Data Format
def ner_combine(ner_pair):
    for idx, (word, ner, cat) in enumerate(ner_pair):
        if (idx + 1) == len(ner_pair) or (idx + 2) == len(ner_pair):
            break
        else:  # 直接將組合詞加入原先分詞好的list (ner_pair)
            n1_ner = ner_pair[idx+1]
            if cat == n1_ner[2]:
                n2_ner = ner_pair[idx+2]
                if ner == 'ORG' and n1_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0],'TITLE+PER',cat)
                    del ner_pair[idx:idx+2]
                    ner_pair.insert(idx,new_wd)     
                elif ner == 'ORG' and n1_ner[1] == 'TITLE' and n2_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0]+n2_ner[0],'ORG+TITLE+PER',cat)
                    del ner_pair[idx:idx+3]
                    ner_pair.insert(idx,new_wd)
                    
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    training_set = []
    trim = stopwords()
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        training_set.append(temp)
#     ner_combine(ner_pair)
    return ner_pair, training_set

In [ ]:
def getLink(google_search_rs):
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
    return getLink

In [ ]:
## Term Frequency Indexing
def target_page(ner_pair):
    df_all_pair = pd.DataFrame(ner_pair,columns=['words','ner','cat'])
########################### dataframe ORG+PER, ORG+TITLE+PER ########################### 
#     df1 = df_all_pair[df_all_pair['ner'] == 'ORG+PER']
#     df2 = df_all_pair[df_all_pair['ner'] == 'ORG+TITLE+PER']
#     df_ppl_pair = pd.concat((df1,df2),ignore_index=True)

########################### only per #################################
    df_ppl_pair = df_all_pair[df_all_pair['ner'] == 'PER']
    cnt = [(c,len(df_ppl_pair[df_ppl_pair['cat']==c]['words'].unique())) for c in df_ppl_pair.cat.unique()]

    link = pd.DataFrame(getLink(google_search_rs), columns=['doc','link'])
    link['cnt'] = 0
    for i in cnt:
        for idx in link.doc:
            if i[0] == idx:
                link['cnt'][idx-1] = i[1]
    link = link.sort_values(by=['cnt'],ascending=False, ignore_index=True)
    link = link.drop(columns=['cnt'])
    return link

In [ ]:
## commands 
search_word = '振华数据'
google_search_rs = pd.read_excel('search3.1.xlsx').url.to_list() # Search2: 振華數據 王雪峰
final_text, getLink = getFinalText(google_search_rs)
ner_pair, training_set = ner_extract(final_text)
print('Target Page of Search 3: "真溱"研究员 (TF of PER)')
target = target_page(ner_pair)
target

In [ ]:
'''
Search 1 「振華數據」；目標：「CEO 王雪峰」
Search 2 「振華數據 王雪峰」；目標：「情報學會」文章 (在第五頁最後一篇)
Search 3 「”真溱”研究員」；目標：「情報理論與實踐」文章 (在第一頁第一篇)
Search 4 「情報理論與實踐」；目標：「期刊網頁」
'''